In [29]:
import pandas as pd
from openpyxl import load_workbook

In [120]:
def header_row_number(sheet):
    for row in sheet.iter_rows(min_row=1, max_col=8):
        if row[1].value and row[1].value == 'Merkki':
            return row[1].row
    return None

COLS = {
    'Käyttöönottovuosi': 'model_year',
    'Käyttööottovuosi': 'model_year',
    'Merkki': 'brand',
    'Malli': 'model',
    'Hylkäys-%': 'fail_pct',
    'Hylkäys %': 'fail_pct',
    "Katsastusten lukumäärä": 'checked_cars',
    "Ajettujen kilometrien keskiarvo": 'driven_avg',
    "Ajettujen kilometrien mediaani": 'driven_med',
    "Yleisin hylkäyssyy": 'reason_1',
    "Toiseksi yleisin hylkäyssyy": 'reason_2',
    "Kolmanneksi yleisin hylkäyssyy": 'reason_3',
}

def should_keep(row):
    w = 'yhteensä'
    return not (w in str(row[0].value) or w in str(row[1].value) or w in str(row[2].value))

def clean_value(c):
    if not c:
        return None
    if isinstance(c.value, str):
        return c.value.replace("\n", " ")
    else:
        return c.value

dfs = []
    
for y in range(2015, 2020):
    print(y)
    fn_xlsx = f'data/Henkilöautojen-määräaikaiskatsastusten-vuositilasto-{y}.xlsx'
    fn_csv = f'data/Henkilöautojen-määräaikaiskatsastusten-vuositilasto-{y}.csv'
    wb = load_workbook(filename = fn_xlsx, read_only=True)
    sheet = wb.active
    header_row = header_row_number(sheet)
    table = [[clean_value(c) for c in r] \
                 for r in sheet.iter_rows(min_row=header_row, max_col=10) \
                     if should_keep(r)]
    
    df = pd.DataFrame(data=table[1:], columns=table[0])
    df.rename(inplace=True, columns=COLS)
    for k in df.keys():
        if k and k not in COLS.values():
            df.drop(inplace=True, columns=[k])
    df.dropna(inplace=True, axis=1, how='all')
    df = df.astype(dtype={'model_year': 'int16', 'brand': 'category', 'model': 'category', 'fail_pct': 'Float16'})
    df.insert(0, 'data_year', y)
    
    df.to_csv(fn_csv, index=False)
    dfs.append(df)

df_all = pd.concat(dfs)
df_all = df_all.astype(errors='ignore', dtype={'checked_cars': 'Int32', 'driven_avg': 'Int32', 'driven_med': 'Int32'})
df_all.to_csv('data/traficom_all.csv', index=False)

print(df_all.info())

2015
2016
2017
2018
2019
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8206 entries, 0 to 1649
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   data_year     8206 non-null   int64  
 1   model_year    8206 non-null   int16  
 2   brand         8206 non-null   object 
 3   model         8206 non-null   object 
 4   fail_pct      8206 non-null   float16
 5   reason_1      7610 non-null   object 
 6   reason_2      7609 non-null   object 
 7   reason_3      7605 non-null   object 
 8   checked_cars  5067 non-null   Int32  
 9   driven_avg    5067 non-null   Int32  
 10  driven_med    5067 non-null   Int32  
dtypes: Int32(3), float16(1), int16(1), int64(1), object(5)
memory usage: 601.0+ KB
None
